In [1]:
import sys
import math
import pandas as pd
import geopandas as gpd

# import custom modules
sys.path.append('../../utils/')
import data_paths
from traffic_counts import TrafficCounts

In [2]:
count = TrafficCounts()


In [3]:
count.get_dtv()

11290.88

In [4]:
count.get_daily_scaling_factor('Local/Collector',"2019-01-02")

14378.135458167331
23708.129562043796


0.61